In [75]:
import pandas as pd
import numpy as np

In [93]:
import requests
from bs4 import BeautifulSoup
import time
from datetime import datetime
import random

In [77]:
from selenium import webdriver
# from selenium.webdriver.common.keys import Keys # don't need this right now
import chromedriver_binary
driver = webdriver.Chrome()

In [78]:
import pickle
import re

# Import CSV of CAS numbers

In [115]:
df = pd.read_csv("~/Desktop/Metis/sea19_ds10/Project04/tsca_inventory_092019_csvf/TSCAINV_092019.csv")

In [117]:
df.tail()

,ID,CASRN,casregno,UID,EXP,ChemName,DEF,UVCB,FLAG,ACTIVITY
68034,68035,2243403-82-7,2243403827,NaN,NaN,"2,5-Furandione, polymer with ethenylbenzene, C...",NaN,UVCB,NaN,ACTIVE
68035,68036,2243403-87-2,2243403872,NaN,NaN,"2,5-Furandione, polymer with ethenylbenzene, C...",NaN,UVCB,NaN,ACTIVE
68036,68037,2245262-16-0,2245262160,NaN,NaN,"2,5-Furandione, polymer with 2-ethyl-2-(hydrox...",NaN,UVCB,PMN; XU,ACTIVE
68037,68038,2250414-82-3,2250414823,NaN,NaN,"Amides, vegetable-oil, N,N-bis(hydroxyethyl), ...",NaN,UVCB,PMN; S,ACTIVE
68038,68039,2304662-68-6,2304662686,NaN,NaN,"9-Octadecenoic acid (9Z)-, reaction products w...",NaN,UVCB,NaN,ACTIVE


In [4]:
casrn = df['CASRN'].copy()

In [5]:
type(casrn), len(casrn), casrn.nunique()

(pandas.core.series.Series, 68039, 68039)

In [59]:
casrn[casrn == "50-24-8"].index[0]

7

## create dictionary of CAS numbers (to later convert to MongoDB)

This step isn't strictly necessary, but it helps me to have worked out the mechanics.

In [91]:
cas_dict = {}

In [65]:
for cas in casrn:
    cas_dict[cas] = ''

In [66]:
cas_dict

{'50-00-0': '',
 '50-01-1': '',
 '50-02-2': '',
 '50-07-7': '',
 '50-14-6': '',
 '50-21-5': '',
 '50-23-7': '',
 '50-24-8': '',
 '50-29-3': '',
 '50-30-6': '',
 '50-32-8': '',
 '50-33-9': '',
 '50-34-0': '',
 '50-45-3': '',
 '50-54-4': '',
 '50-55-5': '',
 '50-69-1': '',
 '50-70-4': '',
 '50-71-5': '',
 '50-78-2': '',
 '50-79-3': '',
 '50-81-7': '',
 '50-84-0': '',
 '50-89-5': '',
 '50-98-6': '',
 '50-99-7': '',
 '51-03-6': '',
 '51-05-8': '',
 '51-17-2': '',
 '51-20-7': '',
 '51-21-8': '',
 '51-28-5': '',
 '51-35-4': '',
 '51-36-5': '',
 '51-43-4': '',
 '51-44-5': '',
 '51-45-6': '',
 '51-46-7': '',
 '51-48-9': '',
 '51-52-5': '',
 '51-55-8': '',
 '51-56-9': '',
 '51-57-0': '',
 '51-64-9': '',
 '51-65-0': '',
 '51-66-1': '',
 '51-74-1': '',
 '51-75-2': '',
 '51-77-4': '',
 '51-78-5': '',
 '51-79-6': '',
 '51-80-9': '',
 '51-82-1': '',
 '51-83-2': '',
 '51-93-4': '',
 '52-01-7': '',
 '52-39-1': '',
 '52-51-7': '',
 '52-52-8': '',
 '52-85-7': '',
 '52-88-0': '',
 '52-89-1': '',
 '52-90-

In [67]:
len(cas_dict)

68039

Done :)

# Scrape PubChem to get URL for a CAS number

Try with Selenium:

In [12]:
# test for the first CASRN in the list: 50-00-0 (formaldehyde)
url = "https://pubchem.ncbi.nlm.nih.gov/#query={}".format("50-00-0")
driver.get(url)

In [13]:
driver.page_source

'<html lang="en"><head>\n    <meta name="robots" content="index,follow,noarchive">\n    <meta charset="UTF-8">\n    <title>PubChem</title>\n    <script type="text/javascript" async="" src="https://www.google-analytics.com/gtm/js?id=GTM-MTC5VWF&amp;t=PubChemTracker&amp;cid=1359622610.1573004089"></script><script async="" src="https://www.google-analytics.com/analytics.js"></script><script type="application/ld+json">\n      {\n          "@context": "https://schema.org",\n          "@type": "Organization",\n          "url": "https://pubchem.ncbi.nlm.nih.gov",\n          "logo": "https://pubchem.ncbi.nlm.nih.gov/pcfe/logo/PubChem_logo.png"\n      }\n    </script>\n    <link rel="apple-touch-icon" sizes="180x180" href="/pcfe/favicon/apple-touch-icon.png">\n    <link rel="icon" type="image/png" sizes="32x32" href="/pcfe/favicon/favicon-32x32.png">\n    <link rel="icon" type="image/png" sizes="16x16" href="/pcfe/favicon/favicon-16x16.png">\n    <link rel="manifest" href="/pcfe/favicon/manifes

Try with BeautifulSoup AND Selenium:

In [14]:
response = requests.get(url)
response

<Response [200]>

In [21]:
# page = response.text
soup = BeautifulSoup(driver.page_source, 'html.parser')

I'm looking for PubChem's "Compound CID" for this molecule, which is given in the url to the compound summary page. Should be within the "COMPOUND BEST MATCH" section.

Should be in this format: https://pubchem.ncbi.nlm.nih.gov/compound/712 where 712 is the CID

In [ ]:
print(soup)

In [22]:
soup.find('a', href=re.compile('^https://pubchem.ncbi.nlm.nih.gov/compound/'))['href']

'https://pubchem.ncbi.nlm.nih.gov/compound/5742'

It works! I got the URL!

Now to do this for all of the CAS numbers: (write the program and then do it in AWS)

First, test the first 5

In [146]:
# write a function 
# that takes: starting CAS, ending CAS, cas_dict, ...?
def get_cid_from_cas(start=0, end=casrn.iloc[-1], cas_dict={}):
    failed_cas = []
    for cas in casrn[start:end]:
        url = "https://pubchem.ncbi.nlm.nih.gov/#query={}".format(cas)
    #     if requests.get(url).status_code == 200:
        try:
            driver.get(url)
            time.sleep(1)
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            cid = soup.find('a', href=re.compile('^https://pubchem.ncbi.nlm.nih.gov/compound/'))['href']
            cas_dict[cas] = cid
            # every few iterations, save cas_dict to a .pickle
            if casrn[casrn == cas].index[0] % 10 == 0:
                print("Saved up to index {}".format(casrn[casrn == cas].index[0]))
                with open('cid_from_cas.pickle', 'wb') as to_write:
                    pickle.dump(cas_dict, to_write)
        except:
            current_time = datetime.now().strftime("%m/%d/%Y, %H:%M:%S")
            status = f"\nFunction: get_cid_from_cas\nDidn't work for {cas} at index {casrn[casrn == cas].index[0]}.\nFailed at {current_time}\n---\n"
            with open('exception_message.txt', 'a') as exception_message:
                exception_message.write(status)
            failed_cas.append(cas)
            continue
    # final save (after loop executes correctly)
    with open('cid_from_cas.pickle', 'wb') as to_write:
        pickle.dump(cas_dict, to_write)
    return cas_dict, failed_cas

In [114]:
# test the "except" message
current_time = datetime.now().strftime("%m/%d/%Y, %H:%M:%S")
print("Current Time =", current_time)
status = f"\nFunction: get_cid_from_cas\nDidn't work for {cas} at index {casrn[casrn == cas].index[0]}.\nFailed at {current_time}\n---\n"
print(status)

Current Time = 11/05/2019, 22:01:33

Function: get_cid_from_cas
Didn't work for 2304662-68-6 at index 68038.
Failed at 11/05/2019, 22:01:33
---



In [99]:
get_cid_from_cas(start=0,end=20)

Saved up to index 0
Saved up to index 10


In [100]:
# with open('cid_from_cas.pickle','rb') as read_file:
#     cas_dict = pickle.load(read_file)
    
cas_dict

{'50-00-0': ['https://pubchem.ncbi.nlm.nih.gov/compound/712'],
 '50-01-1': ['https://pubchem.ncbi.nlm.nih.gov/compound/5742'],
 '50-02-2': ['https://pubchem.ncbi.nlm.nih.gov/compound/5743'],
 '50-07-7': ['https://pubchem.ncbi.nlm.nih.gov/compound/5746'],
 '50-14-6': ['https://pubchem.ncbi.nlm.nih.gov/compound/5280793'],
 '50-21-5': ['https://pubchem.ncbi.nlm.nih.gov/compound/612'],
 '50-23-7': ['https://pubchem.ncbi.nlm.nih.gov/compound/5754'],
 '50-24-8': ['https://pubchem.ncbi.nlm.nih.gov/compound/5755'],
 '50-29-3': ['https://pubchem.ncbi.nlm.nih.gov/compound/3036'],
 '50-30-6': ['https://pubchem.ncbi.nlm.nih.gov/compound/5758'],
 '50-32-8': ['https://pubchem.ncbi.nlm.nih.gov/compound/2336'],
 '50-33-9': ['https://pubchem.ncbi.nlm.nih.gov/compound/4781'],
 '50-34-0': ['https://pubchem.ncbi.nlm.nih.gov/compound/9279'],
 '50-45-3': ['https://pubchem.ncbi.nlm.nih.gov/compound/5765'],
 '50-54-4': ['https://pubchem.ncbi.nlm.nih.gov/compound/441326'],
 '50-55-5': ['https://pubchem.ncbi.nl

In [62]:
casrn[casrn == "50-07-7"].index[0]

3

# Get IUPAC name from CID

In [151]:
cas_dict, failed_cas = get_cid_from_cas(cas_dict={})

TypeError: cannot do slice indexing on <class 'pandas.core.indexes.range.RangeIndex'> with these indexers [2304662-68-6] of <class 'str'>

In [141]:
cas_dict

{'50-00-0': 'https://pubchem.ncbi.nlm.nih.gov/compound/712',
 '50-01-1': 'https://pubchem.ncbi.nlm.nih.gov/compound/5742',
 '50-02-2': 'https://pubchem.ncbi.nlm.nih.gov/compound/5743',
 '50-07-7': 'https://pubchem.ncbi.nlm.nih.gov/compound/5746',
 '50-14-6': 'https://pubchem.ncbi.nlm.nih.gov/compound/5280793'}

In [143]:
for cas in cas_dict:
    print(cas_dict[cas])

https://pubchem.ncbi.nlm.nih.gov/compound/712
https://pubchem.ncbi.nlm.nih.gov/compound/5742
https://pubchem.ncbi.nlm.nih.gov/compound/5743
https://pubchem.ncbi.nlm.nih.gov/compound/5746
https://pubchem.ncbi.nlm.nih.gov/compound/5280793


# Appendix: Potentially useful code pieces

In [ ]:
# How to print the current time:
# now = datetime.now()
# current_time = now.strftime("%H:%M:%S")
# print("Current Time =", current_time)

In [ ]:
# How to use try...except for writing a file that's not writable:
# try:
#   f = open("demofile.txt")
#   f.write("Lorum Ipsum")
# except:
#   print("Something went wrong when writing to the file")
# finally:
#   f.close()

In [ ]:
# How to create a new text file and write to it:
# "x" - Create - will create a file, returns an error if the file exist
# "a" - Append - will create a file if the specified file does not exist
# "w" - Write - will create a file if the specified file does not exist

# f = open("demofile3.txt", "w")
# f.write("Woops! I have deleted the content!")
# f.close()

# OR:

# with open('somefile.txt', 'a') as the_file:
#     the_file.write('Hello\n')

# #open and read the file after the appending:
# f = open("demofile3.txt", "r")
# print(f.read())

In [ ]:
# How to write to a pickle file:
# with open('noise.pickle', 'wb') as to_write:
#     pickle.dump(lots_of_noise, to_write)

In [ ]:
# How to read from a pickle file:
# with open('noise.pickle','rb') as read_file:
#     new_noise = pickle.load(read_file)

In [ ]:
# How to find the index of a pandas series value:
# myseries[myseries == 7].index[0]

In [ ]:
# How to add to a dictionary's value:
# d['word'] = [d['word'],'something']

In [ ]:
# How to click on stuff with Selenium:
# compose_button=driver.find_element_by_xpath('//div[text()="Compose"]')
# compose_button.click()

In [ ]:
# How to enter stuff in a field with Selenium:
# to_field = driver.find_element_by_name("to")
# to_field.send_keys("email@gmail.com") # enter recipient email